# Project 4

## Group Number 6

### Team Members
- Baraa Nidal Ghannoum
- Tom Mantel
- Mubarik Omar Ali Abdi


In [1]:
# Task 0: Install mw_plot library
!pip install mw_plot


In [2]:
import mw_plot
print("mw_plot successfully installed and imported!")


mw_plot successfully installed and imported!
